# Programming Assignment

This programming assignment is migrated from tensorflow 2.0 excercise, aiming at familiar with pytorch programming API

Reference:
* https://nextjournal.com/gkoehler/pytorch-mnist
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
* https://blog.csdn.net/touristourist/article/details/100535544

## CNN classifier for the MNIST dataset

#### The MNIST dataset

In this assignment, you will use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). It consists of a training set of 60,000 handwritten digits with corresponding labels, and a test set of 10,000 images. The images have been normalised and centred. The dataset is frequently used in machine learning research, and has become a standard benchmark for image classification models. 

- Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998.

Your goal is to construct a neural network that classifies images of handwritten digits into one of 10 classes.

In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### Load and preprocess the data

In [ ]:
MNIST_train = datasets.MNIST('./data/minst/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                                 torchvision.transforms.ToTensor()
                             ]))
MNIST_train.data.shape

In [ ]:
MNIST_test = datasets.MNIST('./data/minst/', train=False, download=True,
                            transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor()
                            ]))
MNIST_test.data.shape

In [ ]:
batch_size_train = 256
batch_size_test = 1024

train_loader = DataLoader(
    datasets.MNIST('./data/minst/', train=True, download=True,
                   transform=torchvision.transforms.Compose([
                       torchvision.transforms.ToTensor()
                   ])),
    batch_size=batch_size_train, 
    shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST('./data/minst/', train=False, download=True,
                   transform=torchvision.transforms.Compose([
                       torchvision.transforms.ToTensor()
                   ])),
    batch_size=batch_size_test, 
    shuffle=True
)

In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(f'batch_idx: {batch_idx}, example_data.shape: {example_data.shape},' 
      + f' example_targets.shape: {example_targets.shape}')

In [ ]:
class SequentialNetwork(nn.Module):
    